In [1]:
from usgs import api
import requests 
import json
import utils
import pandas as pd
import json
import satellite_images as si
import os

In [ ]:
URL = "https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/"
username = ''
password = ''
data = {'jsonRequest':'{"username": "'  + username +  '", "password": "'  + password +  '", "authType": "EROS", "catalogId": "EE"}'}
resp  = requests.post(URL+ "login", data=data)
api_key = resp.json()["data"]
print(api_key)

In [ ]:
def find_images_ee(api_key,start,end,lat,lng) :
    print(start,end,lat,lng)
    datasets = [
        'NAIP',
        'HIGH_RES_ORTHO',
        'ORBVIEW3',
        'AVHRR_COMPOSITE',
        'AVHRR_ORBSEG',
        'AVHRR_1K_GLOBAL_COMP',
        'NON_COMMERCIAL_ORDERABLE_IK',
        'LANDSAT_8_C1',
        'LANDSAT_ETM_C1',
        'SPOT_CIB',
    ]
    r= []
    for dataset in datasets : 
        results = api.search(  
                                dataset, 'EE',
                                start_date=start, end_date=end,
                                lat =lat, lng = lng , distance =100 ,
                                api_key=api_key
        )

        for scene in results['data']['results']:
            print(scene,'\n\n')
            r.append(scene)
    
    
    return r


In [2]:
fires = pd.read_csv("./data/Fires_data/incendios2015.csv", sep=';' , encoding="utf-8",na_values="nan")
fires.head()
incendio = fires['incendio']==1
falso_alarme = fires['falso_alarme']==0 
falso_alarme_tipo = fires['tipo']!='Falso Alarme'
fogacho = fires['fogacho']==0
fires = fires[ incendio & falso_alarme & falso_alarme_tipo & fogacho]
fires.drop([
               'ano', 
               'codigo_sgif',
               'codigo_anpc',
               'data_primeira_intervencao',
               'hora_primeira_intervencao',
               'fonte_alerta',
               'nut',
               'ine',
               'causa', 'reacendimento' , 'agricola',
               'queimada','falso_alarme','fogacho','incendio','perimetro','aps', 'regiao_prof', 'ugf' 
          ],axis=1, inplace=True)
fires = fires.sort_values('area_total',  ascending=False)
fires.dropna()
fires.head()

C:\ProgramData\Anaconda3\envs\geo_py37\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,tipo,distrito,concelho,freguesia,local,x,y,lat,lon,data_alerta,hora_alerta,data_extincao,hora_extincao,area_povoamento,area_mato,area_agricola,area_pov_mato,area_total,tipo_causa
21390,Florestal,Viseu,Mangualde,PÃ³voa de CervÃ£es,ContenÃ§as de Baixo,237389.0,398832.0,40:33:26.5068000000082'',7:41:29.922000000001'',06/08/2015,15:59:00,07/08/2015,07:33:00,29.89,955.11,0,985,985,NaN
6551,Florestal,Guarda,MÃªda,Ranhados,Pinhal das Forcas,267527.0,448034.0,40:59:54.643199999993'',7:19:49.9692000000005'',30/06/2015,14:34:00,01/07/2015,00:00:00,55.50,35,7.5,90.5,98,Intencional
7081,Florestal,Viseu,Armamar,SÃ£o Cosmado,Regadas,241299.0,454999.0,41:3:47.069999999992'',7:38:30.4511999999989'',30/03/2015,14:30:00,31/03/2015,02:54:00,24.85,72.89,0,97.74000000000001,97.74,Negligente
3947,Florestal,Castelo Branco,CovilhÃ£,Verdelhos,Sernadas,252322.0,375942.0,40:21:2.06999999999709'',7:31:2.07479999999883'',31/03/2015,09:18:00,01/04/2015,04:04:00,0.00,97.404,0,97.404,97.404,Negligente
21637,Florestal,Braga,GuimarÃ£es,SÃ£o Torcato,BOAVISTA,189451.0,501843.0,41:29:9.29040000000384'',8:15:34.1676000000021'',09/08/2015,11:20:00,10/08/2015,03:33:00,70.00,27,0,97,97,Desconhecida


In [ ]:
head = fires.head(20)
results2015={ }
for index, row in head.iterrows():
    lat =utils.dms_to_decimal(row["lat"],"W")
    lon =utils.dms_to_decimal(row["lon"],"N")
    
    sensing_date_FROM = utils.dmy_to_ymd( row["data_alerta"] )
    sensing_date_TO = utils.dmy_to_ymd(  row["data_extincao"] )
    print('----------------------------------------->>>>>>>>>')
    
    r = find_images_ee(api_key,sensing_date_FROM,sensing_date_TO,lat,lon)
    results2015[index] = r
    print('\n\n')

### Save searches in json 

In [ ]:
results = json.dumps(results2015)
f = open("./data/results2015.json","w")
f.write(results)
f.close()

### import saved searches

In [3]:
with open('./data/results2015.json') as json_file:
    results2015 = json.load(json_file)

## Downloading images

In [3]:
num_imgs = 0
downloadpath = "C:/Users/SimãoPauloLealBarbos/Downloads/"
for index in results2015 :
    if len(results2015[index])>0 :
        print('\n\n',index)
        for result in results2015[index] :
            si.download_landsat_img(index,result) # download img
            print("Donwloading!!")
            num_imgs+=1
            print(result["displayId"])
            browseUrl = result["browseUrl"]
            #si.show_img_from_url(browseUrl) # show natural color full img
            dataAccessUrl = result["dataAccessUrl"]
            downloadUrl = result["downloadUrl"]
            metadataUrl = result["metadataUrl"]
            downloads = utils.wait_for_downloads(60,1)
            outputpath = "../../Satellite_img_2015/" + index
            if (not os.path.isdir(outputpath)):
                os.mkdir(outputpath)
            command_to_move = "mv \"" +  downloadpath + downloads[0] + "\"  " + outputpath
            print("command_to_move",command_to_move)
            os.system( command_to_move )
            
#browseUrl natural img
#dataAccessUrl item basket EE
#downloadUrl url with several downloads options



 21390
LC08_L1TP_203032_20150806_20170406_01_T1
https://earthexplorer.usgs.gov/download/external/options/LANDSAT_8_C1/LC82030322015218LGN01/INVSVC/


 7081
LC08_L1TP_203032_20150331_20170411_01_T1
https://earthexplorer.usgs.gov/download/external/options/LANDSAT_8_C1/LC82030322015090LGN01/INVSVC/
LE07_L1TP_204031_20150330_20161028_01_T1
https://earthexplorer.usgs.gov/download/external/options/LANDSAT_ETM_C1/LE72040312015089NSG00/INVSVC/


 3947
LC08_L1TP_203032_20150331_20170411_01_T1
https://earthexplorer.usgs.gov/download/external/options/LANDSAT_8_C1/LC82030322015090LGN01/INVSVC/


 2653
LE07_L1TP_205031_20150406_20161027_01_T1
https://earthexplorer.usgs.gov/download/external/options/LANDSAT_ETM_C1/LE72050312015096ASN00/INVSVC/


 14918
LE07_L1TP_204031_20150906_20161021_01_T1
https://earthexplorer.usgs.gov/download/external/options/LANDSAT_ETM_C1/LE72040312015249NSG00/INVSVC/


 8093
LE07_L1TP_204031_20150314_20161028_01_T1
https://earthexplorer.usgs.gov/download/external/options/

# Unnzipping tar files and setting up downloaded images

In [6]:
si.unzipping_landsatfiles("../../Satellite_img_2015/")

['7081']
4


# Visualizing images

In [6]:
indexs = os.listdir("../../Satellite_img_2015/")
n=0
#num_imgs=0
for index in indexs :
    path = "../../Satellite_img_2015/"+ index
    images_ids = os.listdir(path) # images founded for index choosen
    for image_id in images_ids :
        n= n + 1 
        print("\n\n->>>>>>>>>>>>>>>>>>", index,"  " ,image_id, "      Image", n ," of " , num_imgs ,"  \n\n " )
        lat = fires.loc[int(index)]["lat"]
        lon = fires.loc[int(index)]["lon"]
        lat =utils.dms_to_decimal(lat,"W")
        lon =utils.dms_to_decimal(lon,"N")
        si.image_true_color_landsat(index,image_id ,"../../Satellite_img_2015/",lon,lat,300)
        